In [103]:
from collections import defaultdict
import json
import os

ARCHIVE_DIR = './archive-Fall2019/'

In [106]:
classes = {}
section_stores = {}
fnames = [ARCHIVE_DIR + f for f in sorted(os.listdir(ARCHIVE_DIR)) if '.json' in f]

for fname in fnames:
    sections = json.load(open(fname))
    section_stores[fname] = {s['Call Number']: s for s in sections}
    for section in sections:
        classes[section['Call Number']] = []
print('Cached data from every date.')

for fname in fnames:
    sections = section_stores[fname]
    for call_num in classes.keys():
        classes[call_num].append( sections.get(call_num, None) )
print('Put all classes into a dictionary by call number.')

courses_indexed = defaultdict(dict)
for c in courses:
    courses_indexed[c['date']][c['Call Number']] = c
print('Indexed all courses by both date and call number.')

Cached data from every date.
Put all classes into a dictionary by call number.
Indexed all courses by both date and call number.


In [11]:
x = json.load(open(ARCHIVE_DIR + f))

2019-08-31: Added 7860.
2019-09-01: Added 7858.
2019-09-02: Added 7856.
2019-09-03: Added 7898.
2019-09-04: Added 7964.
2019-09-05: Added 7967.
2019-09-06: Added 7998.
2019-09-07: Added 7994.
2019-09-08: Added 8001.
2019-09-09: Added 8017.
2019-09-10: Added 8045.
2019-09-11: Added 8057.
2019-09-12: Added 8092.
2019-09-13: Added 8104.
2019-09-14: Added 8105.
2019-09-15: Added 8105.
2019-09-16: Added 8104.
2019-09-17: Added 8106.


In [101]:
added = defaultdict(list)

DATE_INDEX = len(dates) - 1

for call_num in courses_indexed[dates[DATE_INDEX]].keys():
    for date in dates[:DATE_INDEX]:
        added[call_num].append(courses_indexed[date].get(call_num, {}))

reassigned_calls = []
for call_num in added.keys():
    courses_instances = added[call_num]

    # Has always existed in course system
    if any([len(c.keys()) == 0 for c in courses_instances]):
        continue

    if (any([c.get('Location', None) == 'To be announced' for c in courses_instances]) and
        any([c.get('Location', None) != courses_instances[0].get('Location', None) for c in courses_instances])):
            reassigned_calls.append(call_num)

In [102]:
json.dump(
    [c for c in courses if c['Call Number'] in reassigned_calls and c['date'] in dates[:DATE_INDEX]],
    open('./reassigned_calls.json', 'w')
)

In [77]:
added[53898]

[{'number': '001',
  'Call Number': 53898,
  'Points': 3,
  'Day/Time': 'TR 1:10pm-2:25pm',
  'Location': 'To be announced',
  'Enrollment': '49 students (60 max) as of August 31, 2019',
  'Instructor': 'Cathy Popkin',
  'course': 'LITERATURE & EMPIRE (19C LIT)',
  'courseNumber': 'UN3220',
  'subject': 'RUSS',
  'date': '2019-08-31'},
 {'number': '001',
  'Call Number': 53898,
  'Points': 3,
  'Day/Time': 'TR 1:10pm-2:25pm',
  'Location': 'To be announced',
  'Enrollment': '47 students (60 max) as of September  1, 2019',
  'Instructor': 'Cathy Popkin',
  'course': 'LITERATURE & EMPIRE (19C LIT)',
  'courseNumber': 'UN3220',
  'subject': 'RUSS',
  'date': '2019-09-01'},
 {'number': '001',
  'Call Number': 53898,
  'Points': 3,
  'Day/Time': 'TR 1:10pm-2:25pm',
  'Location': '313 Fayerweather',
  'Enrollment': '47 students (60 max) as of September  2, 2019',
  'Instructor': 'Cathy Popkin',
  'course': 'LITERATURE & EMPIRE (19C LIT)',
  'courseNumber': 'UN3220',
  'subject': 'RUSS',
  'd

In [50]:
courses[0]

{'number': '001',
 'Call Number': 14677,
 'Points': 3,
 'Enrollment': '64 students (73 max) as of August 31, 2019',
 'Instructor': 'Trevor S Harris',
 'course': 'Accounting I: Financial Accoun',
 'courseNumber': 'B5001',
 'subject': 'ACCT',
 'date': '2019-08-31'}

In [49]:
firstday_calls

[]

In [46]:
json.dump([c for c in courses if c.get('Location', None) is not None], open('courses.json', 'w'))